In [1]:
from Utils.constants import *
from Utils.scrapping_file import Scraping_Class
from time import sleep
import re

import pandas as pd
from bs4 import BeautifulSoup

import numpy as np

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
headers = requests.utils.default_headers()
headers.update({'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})

In [2]:
Scraping = Scraping_Class()

In [3]:
final_df = pd.DataFrame()

for department in DEPARTMENT_LIST:
    print(f'Departamento: {department}\n===================================||')

    num_of_pages = Scraping.get_num_pages(department)

    for page_num in range(0,num_of_pages):
        print(f'Pagina: {page_num}\n===========||')

        product_list, sku_list = Scraping.get_products_list_page(department, page_num)

        df_cols = ['Departamento','Desconto','Frete','Nome','Preco','Parcelas','SKU']
        products_df = pd.DataFrame(columns=df_cols)

        squareM_flag_list = []
        for product in range(len(product_list)):
            if product % 2 == 0:

                product_format, temp_append = Scraping.get_product_info(product_list, product, department)
                squareM_flag_list.append(temp_append)

                df_to_concat = pd.DataFrame([product_format[1],product_format[2],product_format[3]
                                             ,product_format[4],product_format[5],product_format[6],sku_list[product]['content']]).transpose()
                
                df_to_concat.columns = df_cols
                products_df = pd.concat([products_df, df_to_concat])

        products_df.reset_index(drop=True,inplace=True)
        products_df['Parcelas'] = products_df['Parcelas'].apply(lambda x: x if 'x' in x else 'none')
        
        products_df['Preco'] = products_df['Preco'].apply(lambda x: float(x.replace('.','').replace('R$','').replace(',','.')))
        products_df['Desconto'] = products_df['Desconto'].apply(lambda x: float(x.replace('%','').replace('-','')))

        products_df.reset_index(inplace=True)
        products_df['Preco_desconto'] = products_df['index'].apply(lambda x: round(products_df['Preco'][x] - (products_df['Desconto'][x]/100*products_df['Preco'][x]),2))
        products_df['Metro_quadrado_flag'] = squareM_flag_list

        final_df = pd.concat([products_df,final_df])
        final_df.drop('index',axis=1,inplace=True)

for column in ['Preco_desconto','Desconto']:
    final_df['Preco'] = final_df['Preco'].apply(lambda x: x if x>0 else np.nan)

final_df.reset_index(inplace=True,drop=True)
final_df.drop('Frete',axis=1,inplace=True)


Departamento: decoracao
===================================||
Pagina: 0
===========||
Pagina: 1
===========||
Pagina: 2
===========||
Pagina: 3
===========||
Pagina: 4
===========||
Pagina: 5
===========||
Pagina: 6
===========||
Pagina: 7
===========||
Pagina: 8
===========||
Pagina: 9
===========||
Pagina: 10
===========||
Pagina: 11
===========||
Pagina: 12
===========||
Pagina: 13
===========||
Pagina: 14
===========||
Pagina: 15
===========||
Pagina: 16
===========||
Pagina: 17
===========||
Pagina: 18
===========||
Pagina: 19
===========||
Pagina: 20
===========||
Pagina: 21
===========||
Pagina: 22
===========||
Pagina: 23
===========||
Pagina: 24
===========||
Pagina: 25
===========||
Pagina: 26
===========||
Departamento: fechadura-e-seguranca
===================================||
Pagina: 0
===========||
Pagina: 1
===========||
Pagina: 2
===========||
Pagina: 3
===========||
Pagina: 4
===========||
Pagina: 5
===========||
Pagina: 6
===========||
Pagina: 7
===========||
Pagina:

In [29]:
final_df

,Departamento,Desconto,Nome,Preco,Parcelas,SKU,Preco_desconto,Metro_quadrado_flag
0,Tintas e Acessórios,0.0,Tinta Acrílica Fosca Standard Rende Muito Bran...,383.99,"7x de R$ 54,86",422973,383.99,0
1,Tintas e Acessórios,0.0,Tinta Acrílica Econômica Pintalit Fosca Branco...,129.99,"2x de R$ 65,00",819234,129.99,0
2,Tintas e Acessórios,0.0,Tinta Acrílica Econômica Argalit Fosca Branco ...,119.90,"2x de R$ 59,95",37334,119.90,0
3,Tintas e Acessórios,0.0,Folha de Lixa Massa Ms230 180 Cortag,1.29,none,515660,1.29,0
4,Tintas e Acessórios,14.0,Tinta Acrílica Econômica Argalit Fosca Branco ...,69.99,none,356274,60.19,0
...,...,...,...,...,...,...,...,...
17246,Decoração,60.0,Aplique para Moldura Z1 Clássico Branco 20x20c...,NaN,none,135714,NaN,0
17247,Decoração,0.0,Porta Vela de Vidro Canelado 10cm Cinza Cromus,27.99,none,471949,27.99,0
17248,Decoração,0.0,"Rodaforro em Pvc 2,5cm com 4 Metros Mogno Perm...",46.49,none,136513,46.49,0
17249,Decoração,0.0,"Moldura Colonial em Pvc para Forro 3,8cm com 6...",65.49,none,479016,65.49,0


In [27]:
final_df['Preco'][17250]

numpy.float64